# My First Machine Learning Project: Recyclables Classification

I thought long and hard about my first personal project because I want it to be meaningful and purposeful and also aid in my machine learning learning journey and into computer vision. Everywhere I search there is already a project or a blog written about solving a certain problem, such as identifying and classfying local cuisines, or generating music. After researching hard and long, I have decided to just start and work on recyclables classification despite there having some work done on it as well. It is simple enough for a beginner like myself and it is quite meaningful for the environment.

We will be looking at the dataset provided by [garythung](https://github.com/garythung/trashnet). Since this is my first personal solo project, I do not want to be too ambitious and just want to learn the ropes and process of creating a model. Based on this [news article](https://www.channelnewsasia.com/singapore/what-can-be-recycled-plastic-paper-glass-metal-recycling-bin-box-tips-nea-3100521),  main recyclables in Singapore are glass, metal, paper and plastic, hence we shall aim to classify these 4 recyclables.

# First Look at Data

## Check System Specifications

In [ ]:
# Check System Specs
import sys
import multiprocessing

if __name__ == '__main__':
  print('Python Version')
  print(sys.version)
  print('Version info.')
  print(sys.version_info)
  print('Number of CPU: ', multiprocessing.cpu_count())

## *Import

In [12]:
## Import
import os
import sys
import multiprocessing
import numpy as np
import pandas as pd

#Preprocessing
from PIL import Image
from sklearn.model_selection import train_test_split

#Models
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
!pip install catboost
import catboost
from catboost import CatBoost
!pip install keras-tuner

#Visualization
import matplotlib.pyplot as plt

#Storage
import pyarrow.feather as feather

#Tuning
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score


## *Mount Google Drive

In [ ]:
## Mount Google Drive

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
# Bing Image Downloader
# !pip install bing-image-downloader
# from bing_image_downloader import downloader
# dir = '/content/drive/MyDrive/Colab Notebooks/Recyclables Images'
# downloader.download('paper garbage',limit=5,output_dir = dir,adult_filter_off = False)

In [ ]:
plastic_image_directory = ('/content/gdrive/MyDrive/Garbage Classification/plastic')
# metal_image_directory = ('C:\\Users\\Rui Han\\Desktop\\Coding\\Recyclables Classification\\Garbage Classification\\metal')
image_filename = 'plastic20.jpg'
img = Image.open(os.path.join(metal_image_directory,image_filename))
pixel_array = np.asarray(img)
# print(pixel_array)
print(pixel_array.shape)

#Images have (x,y) pixels up to 255 for RGB
#However, different images have different resolutions

(224, 224, 3)


In [ ]:
print(pixel_array.shape)
hipixel = pixel_array.reshape(len(pixel_array),-1)
print(hipixel.shape)
hidf = pd.DataFrame(hipixel)
display(hidf.tail())

# Data Wrangling Part I: Data Playing

## Choosing Resolution of Images

Each image has different resolution so it is imperative that we standardise the resolution so as to have the same number of features for all. This [site](https://sh-tsang.medium.com/review-learning-to-resize-images-for-computer-vision-tasks-image-classification-image-quality-c05cbe284bc6) suggests a 224x224 so we shall try that first. For the sake of simplicity of our model, we shall do 224x224 for all images, regardless of smaller or bigger.

In [ ]:
image = Image.open(os.path.join(metal_image_directory,image_filename))
resized_image = image.resize((224,224))
resized_image.save('/content/drive/MyDrive/Garbage Classification/metal/test.jpg')

## Extracting Pixel Values as Arrays

In [ ]:
resized_image_array = np.asarray(resized_image)
print(resized_image_array)
print(resized_image_array.shape)

## Visualising Images

In [ ]:
plt.figure(figsize=(10,5))
plt.imshow(resized_image_array)
plt.title('metal_521')
plt.axis('off')
plt.show

# Data Wrangling Part II: Renaming and Resizing Data
There are 3,737 glass images of which 607 are specially named brown glass, 607 are specially named green glass, and 775 are specially named white glass. The rest of 1,726 are named glass but have a mix of all 3 colours.

There are 1,986 metal images, 2,092 paper images and 2,302 plastic images.

I have decided to remove the specially named glass images to keep the number of images for each category similar.

We will first rename all the images for ease of handling.

Then we will resize every image to a size of 224x224.

Did a scan through the images and found that there are some mislabelled images so did the following changes:
```
glass: 1726 > 1725 (glass97)
metal: 1986>1979 (+ 1 from glass97) (metal841 macs box, metal1274 white circle with right arrow, metal1394 chocolate cake, metal1766 macs box, metal1892 cup noodle, metal1900 green instant, metal1934 golden cap, metal1985 cup noodle)
paper: 2092>2088 (paper593 plastic, paper594 plastic, paper597 plastic, paper598 plastic)
plastic: 2303>2306 (+4 from papers)
New Total = 8098
```



In [ ]:
old_dir = '/content/drive/MyDrive/Garbage Classification/raw_metal'
new_dir = old_dir.replace('raw_','')
print(new_dir)

new_filename = old_dir.split('_')[-1] + str(a) + '.jpg'
print(new_filename)

/content/drive/MyDrive/Colab Notebooks/Recyclables Images/metal
metal151.jpg


## *Directories

In [ ]:
## Directories
raw_metal_image_directory = ('/content/gdrive/MyDrive/Garbage Classification/raw_metal')
raw_glass_image_directory = ('/content/gdrive/MyDrive/Garbage Classification/raw_glass')
raw_paper_image_directory = ('/content/gdrive/MyDrive/Garbage Classification/raw_paper')
raw_plastic_image_directory = ('/content/gdrive/MyDrive/Garbage Classification/raw_plastic')
raw_image_directories = [raw_metal_image_directory,raw_glass_image_directory,raw_paper_image_directory,raw_plastic_image_directory]

metal_image_directory = ('/content/gdrive/MyDrive/Garbage Classification/metal')
glass_image_directory = ('/content/gdrive/MyDrive/Garbage Classification/glass')
paper_image_directory = ('/content/gdrive/MyDrive/Garbage Classification/paper')
plastic_image_directory = ('/content/gdrive/MyDrive/Garbage Classification/plastic')
created_image_directories = [
    #metal_image_directory,glass_image_directory,paper_image_directory,
    plastic_image_directory]


## Renaming

In [ ]:
## Renaming

# Delete all created images and empty folders
for directory in created_image_directories:
  dir = directory
  if os.path.exists(os.path.join(dir)):
    for filename in os.listdir(dir):
      os.remove(os.path.join(dir, filename))
  else:
    os.makedirs(dir)

for old_directory in raw_image_directories:
  old_dir = old_directory
  new_dir = old_dir.replace('raw_','')
  a = 1
  for filename in os.listdir(old_dir):
      image = Image.open(os.path.join(old_dir,filename))
      if image.mode != 'RGB':
        image = image.convert('RGB')
      new_filename = old_dir.split('_')[-1] + str(a) + '.jpg'
      a += 1
      image.save(os.path.join(new_dir,new_filename))

print('Renamed successfully!')

## Old code that works
# for dir in created_image_directories:
#   directory = dir
#   a = 1
#   for filename in os.listdir(directory):
#     if filename.startswith(directory.split('/')[-1]):
#       image = Image.open(os.path.join(directory,filename))
#       if image.mode == 'RGBA':
#         image = image.convert('RGB')
#       new_filename = directory.split('/')[-1] + str(a) + '.jpg'
#       a += 1
#       image.save(os.path.join(directory,new_filename))
#       if os.path.exists(os.path.join(directory,filename)):
#         os.remove(os.path.join(directory,filename))

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Renamed successfully!


In [ ]:
# Renaming with 8 CPU Cores

# Delete all created images and empty folders
for directory in created_image_directories:
  dir = directory
  if os.path.exists(os.path.join(dir)):
    for filename in os.listdir(dir):
      os.remove(os.path.join(dir, filename))
  else:
    os.makedirs(dir)

def rename_image(image_path,aa):
  # global aa
  new_dir = image_path.rsplit('/', 1)[0].replace('raw_', '')
  image = Image.open(image_path)
  if image.mode != 'RGB':
    image = image.convert('RGB')
  new_filename = image_path.split('/')[-2].replace('raw_', '')
  aa += 1
  image.save(os.path.join(new_dir,new_filename))


  if __name__ == '__main__':
    raw_image_directories = [raw_metal_image_directory,raw_glass_image_directory,raw_paper_image_directory,raw_plastic_image_directory]

    pool = multiprocessing.Pool(processes=8)

    aa = 0

    for directory in raw_image_directories:
      dir = directory
      image_paths = [os.path.join(dir,filename) for filename in os.listdir(dir)]
      # pool.map(rename_image,image_paths)
      args = [(image_path, aa) for aa, image_path in enumerate(image_paths, start=1)]
      pool.starmap(rename_image, args)

    pool.close()
    pool.join()

print('Renamed successfully!')

Renamed successfully!


In [ ]:
# Check if renaming was done right

for directory in (raw_image_directories):
  dir = directory
  raw_image_count = 0
  for filename in os.listdir(dir):
    if os.path.isfile(os.path.join(dir,filename)):
      raw_image_count += 1
  print(dir.split('_')[-1] + ' count is equal to ' + str(raw_image_count))

for directory in (created_image_directories):
  dir = directory
  created_image_count = 0
  for filename in os.listdir(dir):
    if os.path.isfile(os.path.join(dir,filename)):
      created_image_count += 1
  print(dir.split('/')[-1] + ' count is equal to ' + str(created_image_count))

metal count is equal to 1986
glass count is equal to 1726
paper count is equal to 2092
plastic count is equal to 2302
metal count is equal to 0
glass count is equal to 0
paper count is equal to 0
plastic count is equal to 0


## Resizing

In [ ]:
## Resizing
created_image_directories = [metal_image_directory,glass_image_directory,paper_image_directory,plastic_image_directory]

rd = 64 #resize_dimension
for directory in created_image_directories:
  dir = directory
  for filename in os.listdir(dir):
    image = Image.open(os.path.join(dir,filename))
    new_filename = 'resized_' + filename
    resized_image = image.resize((rd,rd))
    resized_image.save(os.path.join(dir,new_filename))
    os.remove(os.path.join(dir,filename))
    os.rename(os.path.join(dir,new_filename), os.path.join(dir,filename))
print('Resize successful!')

Resize successful!


In [ ]:
## Resizing with 8 CPU cores
import multiprocessing

rd = 64 #resize_dimension
def resize_image(image_path):
  image = Image.open(image_path)
  new_filename = 'resized_' + os.path.basename(image_path)
  resized_image = image.resize((rd,rd))
  resized_image.save(os.path.join(dir,new_filename))
  os.remove(image_path)
  os.rename(os.path.join(dir,new_filename),image_path)

if __name__ == '__main__':
  created_image_directories = [metal_image_directory, glass_image_directory, paper_image_directory, plastic_image_directory]

  pool = multiprocessing.Pool(processes=8)

  for directory in created_image_directories:
    dir = directory
    image_paths = [os.path.join(dir,filename) for filename in os.listdir(dir)]
    pool.map(resize_image,image_paths)

  pool.close()
  pool.join()

print('Resize successfully! ')

Resize successfully! 


In [ ]:
# Check if resizing was done right
created_image_directories = [metal_image_directory,glass_image_directory,paper_image_directory,plastic_image_directory]
for directory in created_image_directories:
  correct_width = rd
  correct_height = rd
  correct_resized_count = 0
  wrong_resized_count = 0
  dir = directory
  for filename in os.listdir(dir):
    image = Image.open(os.path.join(dir,filename))
    width, height = image.size
    if int(width) - correct_width == 0 and int(height) - correct_height == 0:
      correct_resized_count += 1
    else:
      wrong_resized_count += 1
  print(dir.split('/')[-1] + ' correct resized count is equal to ' + str(correct_resized_count))
  print(dir.split('/')[-1] + ' wrong resized count is equal to ' + str(wrong_resized_count))

metal correct resized count is equal to 1986
metal wrong resized count is equal to 0
glass correct resized count is equal to 1726
glass wrong resized count is equal to 0
paper correct resized count is equal to 2092
paper wrong resized count is equal to 0
plastic correct resized count is equal to 2302
plastic wrong resized count is equal to 0


# Data Wrangling Part III: Forming .csv File and Restructuring Data

We will first take each image and extract each pixel as a feature and be put into a Dataframe. There would be a total 8,106 samples each with 150,528(224x224x3), hence the shape would be 8106 x 150528. *change: 8,098 samples*

*At this point I have hit a big wall because the code for forming the dataframe always took super long and at first I thought it was because of my inactivity at first so I tried doing it locally on Jupyter but that took an another day and it was still not done. I guessed it was a computational issue so I decided to rent cloud GPUs but I met many issues with that (such as syncing images to the instance, running the code on the instance) and it wasn't reliable. In the end, I opted for Google Colab pro which would save me all the hassle. I thought the code would be done fast but it has been 19h 30min thus far and so it made me realise that perhaps I would need to downsize the images quite a bit. From this [article](https://hasty.ai/docs/mp-wiki/training-parameters/batch-size#:~:text=%22For%20the%20most%20part%2C%20a,best%20to%20begin%20experimenting%20with.%22) and this [article](https://towardsdatascience.com/boost-your-image-classifier-e1cc7a56b59c), it suggests batch resizing and training the model on increasing dimensional size. For simplicity, I shall start with 32x32, then 64x64, 128x128, etc.*

*I fixed the lengthy time it took and decided to work on the processing speed so I delved into multiprocessing and using the 8 CPU cores that Google Colab has. However, it has been a big failure of .feather files always generating samples from 1 directory or generating 20k samples in 1 .feather file. I decided to just cut out that multiprocessing step for simplicity, since data preprocessing is not the main point of this project.*

We will then create a y variable for each sample.

Then, randomly select 20% of each category to put into a test set. Then create their dataframe of target variable y and 224x224x3 pixel features X.

This will be done for the other 80% of the training set.

## Convert Pixels into .Feather file in Batches

In [ ]:
# Converting Pixels into .Feather 1.0
test = '/content/gdrive/MyDrive/Garbage Classification/test'
test_directories = [test]

num_pixel_columns = rd**2 + 1
num_all_columns = (rd**2 *3)
rawdata = pd.DataFrame() #create empty dataframe
column_names = ['label'] #create column names for each RGB 50176 pixels
red, green, blue = 'R', 'G', 'B'
colours = [red,green,blue]
for colour in colours:
  for i in range(1,num_pixel_columns):
    column_names.append(colour + str(i))

for directory in created_image_directories:
  dir = directory
  label =np.array(dir.split('/')[-1]).reshape(1,-1)
  for filename in os.listdir(dir):
    image = Image.open(os.path.join(dir,filename))
    image_array = np.asarray(image).reshape(-1,num_all_columns)
    image_array = np.concatenate((label,image_array),axis=1)
    rawdata = pd.concat([rawdata,pd.DataFrame(image_array,columns=column_names)],ignore_index=True)
    # rawdata = rawdata.append(pd.DataFrame(image_array,columns=column_names),ignore_index=True)
# rawdata.to_csv('/content/drive/MyDrive/Colab Notebooks/Recyclables Images/rawdata.csv',index=False)
feather.write_feather(rawdata,'/content/drive/MyDrive/Garbage Classification/rawdata.feather')

print('File successfully saved!')
display(rawdata.head())

In [ ]:
# Converting Pixels to .Feather with 8 CPU Cores 2.0

def pixel_to_feather(image_path, label, batch_size, batch_number):
  global rawdata
  image = Image.open(image_path)
  image_array = np.asarray(image).reshape(-1,num_all_columns)
  image_array = np.concatenate((label,image_array),axis=1)
  rawdata = pd.concat([rawdata,pd.DataFrame(image_array,columns=column_names)],ignore_index=True)

  # if len(rawdata) >= len(image_paths) or len(rawdata) >= batch_size:
  #   feather_filename = f'rawdata_batch_{batch_number}.feather'
  #   feather.write_feather(rawdata,f'/content/gdrive/MyDrive/Garbage Classification/{feather_filename}')
  #   print(f'Saved {len(rawdata)} to {feather_filename}')
  #   rawdata = pd.DataFrame(columns=column_names)

  return rawdata

if __name__ == '__main__':
  created_image_directories = [metal_image_directory,glass_image_directory,paper_image_directory,plastic_image_directory]

  rd = 32 #resize_dimension
  num_pixel_columns = rd**2 + 1
  num_all_columns = (rd**2 *3)
  column_names = ['label'] + [f'{colour}{i}' for colour in ['R','G','B'] for i in range(1,num_pixel_columns)]

  rawdata = pd.DataFrame(columns=column_names)
  batch_number = 1

  image_paths = []

  for directory in created_image_directories:
      dir = directory
      label = np.array(dir.split('/')[-1]).reshape(1,-1)
      image_paths += [os.path.join(dir,filename) for filename in os.listdir(dir)]

  batch_size = 2000
  total_samples_in_directory = len(image_paths) #could be more or less than 2000
  diff = total_samples_in_directory - batch_size
  samples_start = 0
  samples_processed = 0
  samples_end = min(batch_size, total_samples_in_directory) #chooses whichever is lower: directory_samples or batch_size

  with multiprocessing.Pool(processes=8) as pool:
    while samples_processed < total_samples_in_directory:
      current_image_paths = image_paths[samples_start:samples_end]
      for image_path in current_image_paths:
        rawdata = pixel_to_feather(image_path,label,batch_size,batch_number)
        samples_processed += 1

      if total_samples_in_directory > batch_size and samples_processed == batch_size:
        feather_filename = f'rawdata_batch_{batch_number}.feather'
        feather.write_feather(rawdata,f'/content/gdrive/MyDrive/Garbage Classification/{feather_filename}')
        print(f'Saved {len(rawdata)} to {feather_filename}')
        rawdata = pd.DataFrame(columns=column_names)
        batch_number += 1

        current_image_paths = image_paths[batch_size + 1:total_samples_in_directory]
        for image_path in current_image_paths:
          rawdata = pixel_to_feather(image_path,label,batch_size,batch_number)
          samples_processed += 1

          if samples_processed == total_samples_in_directory:
            feather_filename = f'rawdata_batch_{batch_number}.feather'
            feather.write_feather(rawdata,f'/content/gdrive/MyDrive/Garbage Classification/{feather_filename}')
            print(f'Saved {len(rawdata)} to {feather_filename}')
            rawdata = pd.DataFrame(columns=column_names)
            batch_number += 1

      if samples_processed == samples_end: #directory_samples <\= batch_size
        feather_filename = f'rawdata_batch_{batch_number}.feather'
        feather.write_feather(rawdata,f'/content/gdrive/MyDrive/Garbage Classification/{feather_filename}')
        print(f'Saved {len(rawdata)} to {feather_filename}')
        rawdata = pd.DataFrame(columns=column_names)
        batch_number += 1

      # if batch_number == 2:
      #   break

print('Files saved successfully!')

Saved 2000 to rawdata_batch_1.feather


In [ ]:
# Converting Pixels to .Feather with 8 CPU Cores 2.1

def pixel_to_feather(image_path, label):
    global rawdata
    image = Image.open(image_path)
    image_array = np.asarray(image).reshape(-1, num_all_columns)
    image_array = np.concatenate((label, image_array), axis=1)
    rawdata = pd.concat([rawdata, pd.DataFrame(image_array, columns=column_names)], ignore_index=True)
    return rawdata

if __name__ == '__main__':
    created_image_directories = [metal_image_directory, glass_image_directory, paper_image_directory, plastic_image_directory]

    rd = 32  # resize_dimension
    num_pixel_columns = rd ** 2 + 1
    num_all_columns = (rd ** 2 * 3)
    column_names = ['label'] + [f'{colour}{i}' for colour in ['R', 'G', 'B'] for i in range(1, num_pixel_columns)]

    rawdata = pd.DataFrame(columns=column_names)
    batch_number = 1
    samples_per_batch = 2000

    with multiprocessing.Pool(processes=8) as pool:
      for directory in created_image_directories:
        dir = directory
        label = np.array(dir.split('/')[-1]).reshape(1, -1)
        image_paths = [os.path.join(dir, filename) for filename in os.listdir(dir)]
        total_samples = len(image_paths)
        batch_start = 0

        while batch_start < total_samples:
            batch_end = min(batch_start + samples_per_batch, total_samples)
            current_image_paths = image_paths[batch_start:batch_end]

            results = pool.starmap(pixel_to_feather, [(image_path,label) for image_path in current_image_paths])
            rawdata = pd.concat(results, ignore_index=True)

            feather_filename = f'rawdata_batch_{batch_number}.feather'
            feather.write_feather(rawdata, f'/content/gdrive/MyDrive/Garbage Classification/{feather_filename}')
            print(f'Saved {len(rawdata)} to {feather_filename}')

            rawdata = pd.DataFrame(columns=column_names)  # Reset rawdata
            batch_start = batch_end
            batch_number += 1

    print('Files saved successfully!')

In [ ]:
## Converting Pixels into .Feather 1.1

def pixel_to_feather(image_path, label):
    global rawdata
    image = Image.open(image_path)
    image_array = np.asarray(image).reshape(-1, num_all_columns)
    image_array = np.concatenate((label, image_array), axis=1)
    rawdata = pd.concat([rawdata, pd.DataFrame(image_array, columns=column_names)], ignore_index=True)
    return rawdata

rd = 224  # resize_dimension
num_pixel_columns = rd ** 2 + 1
num_all_columns = (rd ** 2 * 3)
column_names = ['label'] + [f'{colour}{i}' for colour in ['R', 'G', 'B'] for i in range(1, num_pixel_columns)]

rawdata = pd.DataFrame(columns=column_names)
batch_number = 5
samples_per_batch = 1153

for directory in created_image_directories:
  dir = directory
  label = np.array(dir.split('/')[-1]).reshape(1, -1)
  image_paths = [os.path.join(dir, filename) for filename in os.listdir(dir)]
  total_samples = len(image_paths)
  batch_start = 0

  while batch_start < total_samples:
      batch_end = min(batch_start + samples_per_batch, total_samples)
      current_image_paths = image_paths[batch_start:batch_end]

      for image_path in current_image_paths:
        data = pixel_to_feather(image_path,label)

      feather_filename = f'rawdata_batch_{batch_number}.feather'
      feather.write_feather(rawdata, f'/content/gdrive/MyDrive/Garbage Classification/{feather_filename}')
      print(f'Saved {len(rawdata)} to {feather_filename}')

      rawdata = pd.DataFrame(columns=column_names)  # Reset rawdata
      batch_start = batch_end
      batch_number += 1

print('Files saved successfully!')

FileNotFoundError: ignored

In [ ]:
try:
    with open('/content/gdrive/MyDrive/Garbage Classification/plastic/plastic20.jpg') as file:
        pass
    print("File found.")
except FileNotFoundError:
    print("File not found.")

File found.


In [ ]:
# Viewing data

df1 = feather.read_feather('/content/gdrive/MyDrive/Garbage Classification/rawdata_batch_1.feather')
df2 = feather.read_feather('/content/gdrive/MyDrive/Garbage Classification/rawdata_batch_2.feather')
df3 = feather.read_feather('/content/gdrive/MyDrive/Garbage Classification/rawdata_batch_3.feather')
df4 = feather.read_feather('/content/gdrive/MyDrive/Garbage Classification/rawdata_batch_4.feather')
df5 = feather.read_feather('/content/gdrive/MyDrive/Garbage Classification/rawdata_batch_5.feather')
df6 = feather.read_feather('/content/gdrive/MyDrive/Garbage Classification/rawdata_batch_6.feather')

for df in [df1,df2,df3,df4,df5,df6]:
  print(df.shape)
  display(df.head())

## *Read file

In [ ]:
## Read and turn into one dataframe
df1 = feather.read_feather('/content/gdrive/MyDrive/Garbage Classification/rawdata_batch_1.feather')
df2 = feather.read_feather('/content/gdrive/MyDrive/Garbage Classification/rawdata_batch_2.feather')
df3 = feather.read_feather('/content/gdrive/MyDrive/Garbage Classification/rawdata_batch_3.feather')
df4 = feather.read_feather('/content/gdrive/MyDrive/Garbage Classification/rawdata_batch_4.feather')
df5 = feather.read_feather('/content/gdrive/MyDrive/Garbage Classification/rawdata_batch_5.feather')
df6 = feather.read_feather('/content/gdrive/MyDrive/Garbage Classification/rawdata_batch_6.feather')

dfs = [df1,df2,df3,df4,df5,df6]
data = pd.concat(dfs,axis=0)
data.reset_index(drop=True,inplace=True)
print(data.shape)
display(data.head())

(8106, 12289)


,label,R1,R2,R3,R4,R5,R6,R7,R8,R9,...,B4087,B4088,B4089,B4090,B4091,B4092,B4093,B4094,B4095,B4096
0,metal,255,255,255,255,255,255,255,255,255,...,255,254,255,255,252,255,255,252,255,255
1,metal,253,253,253,253,253,253,253,253,253,...,253,253,253,253,253,253,253,253,253,253
2,metal,220,221,239,220,221,239,219,220,238,...,169,159,159,171,162,162,172,165,165,175
3,metal,159,112,60,163,115,66,170,118,71,...,60,104,72,51,91,60,42,82,53,35
4,metal,126,93,62,129,96,65,134,99,69,...,196,197,191,193,195,186,187,192,183,184


In [ ]:
display(data[data['label'] == 'metal'])

,label,R1,R2,R3,R4,R5,R6,R7,R8,R9,...,B4087,B4088,B4089,B4090,B4091,B4092,B4093,B4094,B4095,B4096
0,metal,255,255,255,255,255,255,255,255,255,...,255,254,255,255,252,255,255,252,255,255
1,metal,253,253,253,253,253,253,253,253,253,...,253,253,253,253,253,253,253,253,253,253
2,metal,220,221,239,220,221,239,219,220,238,...,169,159,159,171,162,162,172,165,165,175
3,metal,159,112,60,163,115,66,170,118,71,...,60,104,72,51,91,60,42,82,53,35
4,metal,126,93,62,129,96,65,134,99,69,...,196,197,191,193,195,186,187,192,183,184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1981,metal,255,255,255,255,255,255,255,255,255,...,253,254,254,254,255,255,255,255,255,255
1982,metal,114,115,119,126,127,131,124,125,129,...,4,7,3,4,7,3,4,7,3,4
1983,metal,221,221,219,221,221,219,220,220,218,...,232,238,234,233,238,234,233,239,235,234
1984,metal,255,255,255,255,255,255,253,255,254,...,238,254,255,255,250,254,253,252,255,255


## *Train Test Splitting and Integer Encoding

In [ ]:
## Split into stratified train and test sets and Integer Encoding Labels into Integers
label_mapping = {'metal': 0, 'glass': 1, 'paper': 2, 'plastic': 3}
data['encoded_label'] = data['label'].map(label_mapping)
columns_to_drop = ['label','encoded_label']

y = data['encoded_label']
X = data.drop(columns=columns_to_drop,axis=1)
X = X.astype('float32') / 255.0

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,stratify=y,random_state=0)
print(f'X_train shape is: {X_train.shape}')
print(f'y_train shape is: {y_train.shape}')
print(f'X_test shape is: {X_test.shape}')
print(f'y_test shape is: {y_test.shape}')

X_train shape is: (6484, 12288)
y_train shape is: (6484,)
X_test shape is: (1622, 12288)
y_test shape is: (1622,)


# Modelling Using ML Models

## *One-hot encoding Function

In [ ]:
## One hot encoding for y_pred and y_test
def ohe_y(y):
  n_classes = 4
  y_reshaped = np.zeros((len(y),n_classes))
  y_reshaped[np.arange(len(y)),y] = 1
  return y_reshaped

## Modelling Part I: KNN

Results:

32x32: 0.60543,0.73652,0.60526

64x64: 0.59864, 0.73221,0.60058

In [ ]:
# KNN

model = KNeighborsClassifier(n_neighbors=5)
model.fit(X_train,y_train)
y_pred = (model.predict(X_test)).astype(int).flatten()

accu_knn = accuracy_score(ohe_y(y_test),ohe_y(y_pred))
roc_knn = roc_auc_score(ohe_y(y_test),ohe_y(y_pred),average='weighted',multi_class='ovo')
f1_knn = f1_score(ohe_y(y_test),ohe_y(y_pred),average='weighted')

print('Accuracy score is ', str(format(accu_knn,'.5f')))
print('ROC AUC score is ', str(format(roc_knn,'.5f')))
print('F1 score is ', str(format(f1_knn,'.5f')))

Accuracy score is  0.59864
ROC AUC score is  0.73221
F1 score is  0.60058


## Modelling Part II: SVC

Results:

32x32: 0.59125,0.72865,0.59154

64x64: 0.66954,0.78006,0.67030

In [ ]:
# SVC

model = SVC(kernel='linear',C=1,
            # decision_function='ovo'
            )
model.fit(X_train,y_train)
y_pred = (model.predict(X_test)).astype(int).flatten()

accu_svc = accuracy_score(ohe_y(y_test),ohe_y(y_pred))
roc_svc = roc_auc_score(ohe_y(y_test),ohe_y(y_pred),average='weighted',multi_class='ovo')
f1_svc = f1_score(ohe_y(y_test),ohe_y(y_pred),average='weighted')

print('Accuracy score is ', str(format(accu_svc,'.5f')))
print('ROC AUC score is ', str(format(roc_svc,'.5f')))
print('F1 score is ', str(format(f1_svc,'.5f')))

Accuracy score is  0.66954
ROC AUC score is  0.78006
F1 score is  0.67030


## Modelling Part III: Catboost

Results:

32x32: 0.82491,0.88258,0.82421

64x64: 0.82614,0.88319,0.82554

In [ ]:
# Catboost

model = catboost.CatBoostClassifier(loss_function='MultiClass',one_hot_max_size=4,random_state=0,verbose=False)
model.fit(X_train,y_train)
y_pred = (model.predict(X_test)).astype(int).flatten()

accu_cb = accuracy_score(ohe_y(y_test),ohe_y(y_pred))
roc_cb = roc_auc_score(ohe_y(y_test),ohe_y(y_pred),average='weighted',multi_class='ovo')
f1_cb = f1_score(ohe_y(y_test),ohe_y(y_pred),average='weighted')

print('Accuracy score is ', str(format(accu_cb,'.5f')))
print('ROC AUC score is ', str(format(roc_cb,'.5f')))
print('F1 score is ', str(format(f1_cb,'.5f')))

Accuracy score is  0.82614
ROC AUC score is  0.88319
F1 score is  0.82554


# Modelling using Deep Learning



## *Reshape Data for CNN

In [ ]:
rd = 64 #resize_dimension

X_train_pixel_data = X_train.values
X_test_pixel_data = X_test.values
X_train_reshaped = (X_train_pixel_data.reshape(-1,rd,rd,3)).astype('float32')
X_test_reshaped = (X_test_pixel_data.reshape(-1,rd,rd,3)).astype('float32')
print(X_train_reshaped.shape)
print(X_test_reshaped.shape)

hi = ohe_y(y_train)
ho = ohe_y(y_test)

(6484, 64, 64, 3)
(1622, 64, 64, 3)


## Simple CNN Architecture 1.1

For starters, I used a simple CNN architecture which is used for identifying and classifying images. Using tensorflow, I created a 5 layer neural network.

The first 3 layers consist of 3 layers that combine a convolutional layer and a pooling layer. The convolutional layer serves to process input data and identify simple patterns such as edges or gradients in the images. The pooling layer then retains the most important information while reducing the spatial dimensions of the output.

The 4th layer flattens the output and turns into a 1D vector that can be fed into the next few layers.

The 5th layer is a dense layer that focuses on making predictions from the output.

The final output layer outputs the predicted class probabilities and then chooses the one with the highest probability as its predicted label.

The final values from fitting the model are:


```
32x32: loss: 0.1878 - accuracy: 0.9337 - val_loss: 1.5473 - val_accuracy: 0.7244

64x64:loss: 0.1364 - accuracy: 0.9664 - val_loss: 2.8081 - val_accuracy: 0.6905 (best was 0.7355)
```




We can see that the train accuracy is high and close to 1 but the validation accuracy is far from that. This hints that the model is fitting well to the train data but not the test data, which means that our model may have a high variance issue.

Furthermore, increasing image dimensions actually decreases the val accuracy which means the model is highly overfitted. For that we shall tweak the architecture a bit.

In [ ]:
#Simple CNN Architecture 1.1

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.activations import relu, linear
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam


model = models.Sequential()

#Convolutional Layer 1
model.add(layers.Conv2D(16,(3,3),activation='relu',input_shape=(rd,rd,3)))
model.add(layers.MaxPooling2D((2,2)))

#Convolutional Layer 2
model.add(layers.Conv2D(32,(3,3),activation='relu'))
model.add(layers.MaxPooling2D((2,2)))

#Convolutional Layer 3
model.add(layers.Conv2D(32,(3,3),activation='relu'))

#Layer 4 Flatten
model.add(layers.Flatten())

#Layer 5 Fully connected
model.add(layers.Dense(32,activation='relu'))
model.add(layers.Dense(4,activation='softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

model.fit(x=X_train_reshaped,
          y=hi,
          batch_size=32,
          epochs=50,
          validation_data=(X_test_reshaped,ho),
          )

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 62, 62, 16)        448       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 31, 31, 16)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 29, 29, 32)        4640      
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 14, 14, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 12, 12, 32)        9248      
                                                                 
 flatten_1 (Flatten)         (None, 4608)             

## Simple CNN Architecture 1.2

### High Variance and Regularization

For tweaking our architecture, we shall focus on reducing overfitting and getting rid of high variance.

Firstly, we shall reduce model's complexity by lowering the number of filters in each convolutional layer. We shall cut each layer's filters by half.We shall do this for the dense layer neurons as well.

Next, we shall add a dropout layer after the first dense layer, whcih randomly deactivates the neurons of this dense layer to prevent overfitting by preventing reliance on any particular set of neurons. We will set 0.5 as the fraction of neurons to drop for starters

For the third tweak, we would be adding early stopping to monitor models' validation accuracy and stopping the model when it no longer shows improvement, to find the optimal epoch number and prevent overfitting. We shall start with a small patience value at 10 for now.

Next, we will be extending the number of epochs since we are implementing many regularization techniques.

`Epoch 28 64x64: loss: 1.3811 - accuracy: 0.2839 - val_loss: 1.3809 - val_accuracy: 0.2842`

Seems like we have regularized too much. Let us reset the number of filters and dense layer neurons. We will keep dropout and early stopping and epoch number.

In [ ]:
#Simple CNN Architecture 1.2

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.activations import relu, linear
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss',
                               patience=10,
                               restore_best_weights=True)

model = models.Sequential()

#Convolutional Layer 1
model.add(layers.Conv2D(8,(3,3),activation='relu',input_shape=(rd,rd,3)))
model.add(layers.MaxPooling2D((2,2)))

#Convolutional Layer 2
model.add(layers.Conv2D(16,(3,3),activation='relu'))
model.add(layers.MaxPooling2D((2,2)))

#Convolutional Layer 3
model.add(layers.Conv2D(16,(3,3),activation='relu'))

#Layer 4 Flatten
model.add(layers.Flatten())

#Layer 5 Fully connected
model.add(layers.Dense(16,activation='relu'))
model.add(layers.Dropout(0.5))

model.add(layers.Dense(4,activation='softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

model.fit(x=X_train_reshaped,
          y=hi,
          batch_size=32,
          epochs=100,
          validation_data=(X_test_reshaped,ho),
          callbacks=[early_stopping]
          )

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 62, 62, 8)         224       
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 31, 31, 8)         0         
 g2D)                                                            
                                                                 
 conv2d_10 (Conv2D)          (None, 29, 29, 16)        1168      
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 14, 14, 16)        0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 12, 12, 16)        2320      
                                                                 
 flatten_3 (Flatten)         (None, 2304)             

## Simple CNN Architecture 1.3

### Choosing Optimal Dropout Fraction
`Epoch 20 64x64:loss: 1.3811 - accuracy: 0.2839 - val_loss: 1.3809 - val_accuracy: 0.2842`

No change at all. We shall temporarily switch off the dropout layer and see whether it's because it's too big.

`Epoch 17 64x64 loss: 0.2398 - accuracy: 0.9133 - val_loss: 1.7411 - val_accuracy: 0.6726`

We will now try different fractions of dropout and pick the best one:

```
0.0: loss: 0.2398 - accuracy: 0.9133 - val_loss: 1.7411 - val_accuracy: 0.6726

0.1: loss: 0.3475 - accuracy: 0.8612 - val_loss: 0.9990 - val_accuracy: 0.7022

0.2: loss: 1.3812 - accuracy: 0.2839 - val_loss: 1.3809 - val_accuracy: 0.2842

0.15: loss: 0.2624 - accuracy: 0.8927 - val_loss: 1.2451 - val_accuracy: 0.7324

0.175: loss: 1.3812 - accuracy: 0.2839 - val_loss: 1.3809 - val_accuracy: 0.284

0.1625: loss: 0.4614 - accuracy: 0.8117 - val_loss: 1.1683 - val_accuracy: 0.7145

0.15625: loss: 0.3273 - accuracy: 0.8695 - val_loss: 1.1305 - val_accuracy: 0.6930

0.16875: loss: 0.2978 - accuracy: 0.8871 - val_loss: 1.2931 - val_accuracy: 0.7232

0.165625: loss: 0.3475 - accuracy: 0.8712 - val_loss: 1.2771 - val_accuracy: 0.6954
```



0.1625 dropout fraction seems to be the best due to the smallest difference between accuracies as well as being high relatively values. Also, the validation loss is on the better end. 0.15625 is a big contender as well but a higher dropout fraction is better to combat overfitting.

In [ ]:
#Simple CNN Architecture 1.3

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.activations import relu, linear
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss',
                               patience=10,
                               restore_best_weights=True)

model = models.Sequential()

#Convolutional Layer 1
model.add(layers.Conv2D(16,(3,3),activation='relu',input_shape=(rd,rd,3)))
model.add(layers.MaxPooling2D((2,2)))

#Convolutional Layer 2
model.add(layers.Conv2D(32,(3,3),activation='relu'))
model.add(layers.MaxPooling2D((2,2)))

#Convolutional Layer 3
model.add(layers.Conv2D(32,(3,3),activation='relu'))

#Layer 4 Flatten
model.add(layers.Flatten())

#Layer 5 Fully connected
model.add(layers.Dense(32,activation='relu'))
model.add(layers.Dropout(0.1625))

# Layer 6 Prediction
model.add(layers.Dense(4,activation='softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

model.fit(x=X_train_reshaped,
          y=hi,
          batch_size=32,
          epochs=100,
          validation_data=(X_test_reshaped,ho),
          callbacks=[early_stopping]
          )

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_39 (Conv2D)          (None, 62, 62, 16)        448       
                                                                 
 max_pooling2d_26 (MaxPooli  (None, 31, 31, 16)        0         
 ng2D)                                                           
                                                                 
 conv2d_40 (Conv2D)          (None, 29, 29, 32)        4640      
                                                                 
 max_pooling2d_27 (MaxPooli  (None, 14, 14, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_41 (Conv2D)          (None, 12, 12, 32)        9248      
                                                                 
 flatten_13 (Flatten)        (None, 4608)            

## Simple CNN Architecture 1.4

### Increasing Model Complexity

Now that we have cleared the high variance issue with dropout and early stopping, we have to deal with increasing the train and validation accuracies by increasing the model's complexity. For starters, we shall double convolutional filters and dense neurons.
```
32,32,64,64,64: loss: 1.3812 - accuracy: 0.2839 - val_loss: 1.3809 - val_accuracy: 0.2842
32,32,64,128,64: loss: 0.1534 - accuracy: 0.9459 - val_loss: 1.4052 - val_accuracy: 0.7417
32,32,64,128,64: loss: 0.2271 - accuracy: 0.9206 - val_loss: 1.1634 - val_accuracy: 0.7398
32,32,64p,64,64: loss: 0.3224 - accuracy: 0.8828 - val_loss: 0.9715 - val_accuracy: 0.7497

32,32,64p,128p,64: loss: 0.1890 - accuracy: 0.9321 - val_loss: 1.1251 - val_accuracy: 0.7700
32,32,64p,128p,64,DF0.325: loss: 0.3251 - accuracy: 0.8794 - val_loss: 0.8978 - val_accuracy: 0.7731
32,64,64p,128p,64,DF0.4 loss: 0.2900 - accuracy: 0.8970 - val_loss: 0.9962 - val_accuracy: 0.7855
32,64p,64p,128p,64,DF0.5: loss: 1.3812 - accuracy: 0.2839 - val_loss: 1.3809 - val_accuracy: 0.2842
32p,64p,64p,128p,64,DF0.45: loss: 0.3017 - accuracy: 0.8899 - val_loss: 0.8716 - val_accuracy: 0.7805
32p,64p,64p,128p,64,DF0.425: loss: 0.2627 - accuracy: 0.9035 - val_loss: 0.9003 - val_accuracy: 0.7454
```
0.4 and 0.45 dropout fraction seems promising. Now we shall try with batch normalisation.



```
32,32,64p,64,64: loss: 0.0833 - accuracy: 0.9696 - val_loss: 0.9244 - val_accuracy: 0.7805
32p,32p,64p,128p,64: loss: 0.1430 - accuracy: 0.9477 - val_loss: 0.9373 - val_accuracy: 0.7848
32p,64p,64p,128p,64,DF0.325: loss: 0.0591 - accuracy: 0.9795 - val_loss: 0.8512 - val_accuracy: 0.7965
32p,64p,64p,128p,64,DF0.4: loss: 0.0895 - accuracy: 0.9692 - val_loss: 1.0204 - val_accuracy: 0.7540
32p,64p,64p,128p,64,DF0.5: loss: 0.1088 - accuracy: 0.9624 - val_loss: 0.8020 - val_accuracy: 0.7811
32p,64p,64p,128p,64,DF0.45: loss: 0.0833 - accuracy: 0.9716 - val_loss: 1.1734 - val_accuracy: 0.7244
32p,64p,64p,128p,64,DF0.425: loss: 0.0855 - accuracy: 0.9718 - val_loss: 0.8041 - val_accuracy: 0.8033
```

0.425 DF seems to work the best so we'll go with that for now.






In [ ]:
#Simple CNN Architecture 1.4

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.activations import relu, linear
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import BatchNormalization

early_stopping = EarlyStopping(monitor='val_loss',
                               patience=10,
                               restore_best_weights=True)

model = models.Sequential()

#Convolutional Layer 1
model.add(layers.Conv2D(32,(3,3),activation='relu',input_shape=(rd,rd,3)))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D((2,2)))

#Convolutional Layer 2
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D((2,2)))

#Convolutional Layer 3
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D((2,2)))

#Convolutional Layer 4
model.add(layers.Conv2D(128,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D((2,2)))

#Layer 5 Flatten
model.add(layers.Flatten())

#Layer 6 Fully connected
model.add(layers.Dense(64,activation='relu'))
model.add(BatchNormalization())
model.add(layers.Dropout(0.425))

# Layer 7 Prediction
model.add(layers.Dense(4,activation='softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

model.fit(x=X_train_reshaped,
          y=hi,
          batch_size=32,
          epochs=100,
          validation_data=(X_test_reshaped,ho),
          callbacks=[early_stopping]
          )

Model: "sequential_31"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_109 (Conv2D)         (None, 62, 62, 32)        896       
                                                                 
 batch_normalization_35 (Ba  (None, 62, 62, 32)        128       
 tchNormalization)                                               
                                                                 
 max_pooling2d_88 (MaxPooli  (None, 31, 31, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_110 (Conv2D)         (None, 29, 29, 32)        9248      
                                                                 
 batch_normalization_36 (Ba  (None, 29, 29, 32)        128       
 tchNormalization)                                               
                                                     

## Simple CNN Architecture 1.5

# Data Augmentation

We will now use data augmentation to increase the amount of data that we have, to try and mitigate the high variance issue that we are facing.

```
No DA: loss: 0.0855 - accuracy: 0.9718 - val_loss: 0.8041 - val_accuracy: 0.8033
DA: loss: 0.5262 - accuracy: 0.8054 - val_loss: 0.5846 - val_accuracy: 0.7744
```

Data augmentation significantly worsened our train loss and accuracy but our validation loss became better and the difference between both accuracies have a gap of only 0.3!!! So now we have fixed our variance issue it is time to improve the complexity once again.

In [ ]:
#Simple CNN Architecture 1.5

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.activations import relu, linear
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

datagen.fit(X_train_reshaped)


#Early Stopping
early_stopping = EarlyStopping(monitor='val_loss',
                               patience=10,
                               restore_best_weights=True)

model = models.Sequential()

#Convolutional Layer 1
model.add(layers.Conv2D(32,(3,3),activation='relu',input_shape=(rd,rd,3)))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D((2,2)))

#Convolutional Layer 2
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D((2,2)))

#Convolutional Layer 3
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D((2,2)))

#Convolutional Layer 4
model.add(layers.Conv2D(128,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D((2,2)))

#Layer 5 Flatten
model.add(layers.Flatten())

#Layer 6 Fully connected
model.add(layers.Dense(64,activation='relu'))
model.add(BatchNormalization())
model.add(layers.Dropout(0.425))

# Layer 7 Prediction
model.add(layers.Dense(4,activation='softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

model.fit(datagen.flow(x=X_train_reshaped,y=hi,batch_size=32),
          epochs=100,
          validation_data=(X_test_reshaped,ho),
          callbacks=[early_stopping]
          )

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 62, 62, 32)        896       
                                                                 
 batch_normalization_5 (Bat  (None, 62, 62, 32)        128       
 chNormalization)                                                
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 31, 31, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 batch_normalization_6 (Bat  (None, 29, 29, 64)        256       
 chNormalization)                                                
                                                      

## Keras Tuning

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.activations import relu, linear
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
from tensorflow.keras import regularizers

class MyImageClassifierTuner:
  def __init__(self,X_train,y_train,X_test,y_test):
    self.X_train = X_train_reshaped
    self.y_train = hi
    self.X_test = X_test_reshaped
    self.y_test = ho
    self.tuner = None

  def build_model(self,hp):
    model = models.Sequential()

    ## Hyperparameter setting for Filter and Kernel
    num_filters = hp.Int('num_filters',min_value=32,max_value=128,step=32)
    kernel_size = hp.Int('kernel_size',min_value=3,max_value=5)

    #Convolutional Layer 1
    model.add(layers.Conv2D(num_filters,(kernel_size,kernel_size),activation='relu',input_shape=(rd,rd,3)))
    model.add(BatchNormalization())
    model.add(layers.MaxPooling2D((2,2)))

    #Convolutional Layer 2,3,4
    for _ in range(3):
      model.add(layers.Conv2D(num_filters,(kernel_size,kernel_size),activation='relu',padding='same'))
      model.add(BatchNormalization())
      model.add(layers.MaxPooling2D((2,2)))

    #Flatten Layer 5
    model.add(layers.Flatten())

    ## Hyperparameter setting for Dense Neurons and Layers
    dense_neurons = hp.Int('dense_neurons',min_value=32,max_value=160,step=32)
    num_hidden_layers = hp.Int('num_hidden_layers',min_value=1,max_value=3)

    #Fully Connected Dense Layers (6-8)
    for _ in range(num_hidden_layers):
      ## Hyperparameter setting for l1 l2 regularizers
      model.add(Dense(dense_neurons,activation='relu',
                      kernel_regularizer=regularizers.l1_l2(
                          l1=hp.Float('l1_regularization',min_value=1e-6,max_value=1e-3,sampling='log'),
                          l2=hp.Float('l2_regularization',min_value=1e-6,max_value=1e-3,sampling='log')
                      )))
      model.add(BatchNormalization())
      model.add(layers.Dropout(0.425))

    # Prediction Layer
    model.add(layers.Dense(4,activation='softmax'))

    ## Hyperparameter setting for learning rate
    model.compile(optimizer=Adam(learning_rate=hp.Float("learning_rate", min_value=1e-4, max_value=1e-2, sampling="log")),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

  ## Max_trials
  def run_tuner(self):
    self.tuner = RandomSearch(
        self.build_model,
        objective='val_accuracy',
        max_trials=20, #Number of diff hyperparameter combinations to try
        directory='my_tuner_directory', #Directory to store tuner logs and checkpoints
        project_name='my_image_classifier', #Name of tuning project
        )

    datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    datagen.fit(X_train_reshaped)
    early_stopping = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)


    self.tuner.search(datagen.flow(x=self.X_train,y=self.y_train,batch_size=32),
                 epochs=100,
                 validation_data=(self.X_test,self.y_test),
                 callbacks=[early_stopping])

    best_hps = self.tuner.get_best_hyperparameters(num_trials=1)[0]

    best_model = self.tuner.hypermodel.build(best_hps)

    best_model.fit(datagen.flow(x=self.X_train,y=self.y_train,batch_size=32),
                 epochs=100,
                 validation_data=(self.X_test,self.y_test),
                 callbacks=[early_stopping])

  ##Get best hyperparameters
  def get_best_hyperparameters(self):
    best_hps = self.tuner.get_best_hyperparameters()[0]

    best_num_filters = best_hps.get('num_filters')
    best_kernel_size = best_hps.get('kernel_size')
    best_dense_neurons = best_hps.get('dense_neurons')
    best_num_hidden_layers = best_hps.get('num_hidden_layers')
    best_l1_regularization = best_hps.get('l1_regularization')
    best_l2_regularization = best_hps.get('l2_regularization')
    best_learning_rate = best_hps.get('learning_rate')

    return (best_num_filters, best_kernel_size, best_dense_neurons, best_num_hidden_layers, best_l1_regularization, best_l2_regularization, best_learning_rate)



tuner = MyImageClassifierTuner(X_train_reshaped,hi,X_test_reshaped,ho)
tuner.run_tuner()

## Obtain best hyperparameters
best_hyperparameters = tuner.get_best_hyperparameters()
print("Best Hyperparameters:")
print(f"Number of Filters: {best_hyperparameters[0]}")
print(f"Kernel Size: {best_hyperparameters[1]}")
print(f"Dense Neurons: {best_hyperparameters[2]}")
print(f"Number of Hidden Layers: {best_hyperparameters[3]}")
print(f"L1 Regularization: {best_hyperparameters[4]}")
print(f"L2 Regularization: {best_hyperparameters[5]}")
print(f"Learning Rate: {best_hyperparameters[6]}")

Trial 13 Complete [00h 58m 38s]
val_accuracy: 0.76325523853302

Best val_accuracy So Far: 0.8162761926651001
Total elapsed time: 05h 05m 45s

Search: Running Trial #14

Value             |Best Value So Far |Hyperparameter
96                |64                |num_filters
3                 |3                 |kernel_size
128               |96                |dense_neurons
2                 |1                 |num_hidden_layers
1.4261e-05        |0.0001142         |l1_regularization
0.00067424        |1.1324e-05        |l2_regularization
0.0001552         |0.00025845        |learning_rate

Epoch 1/100
203/203 [==============================] - 53s 247ms/step - loss: 2.1079 - accuracy: 0.3544 - val_loss: 1.9035 - val_accuracy: 0.2676
Epoch 2/100
203/203 [==============================] - 49s 242ms/step - loss: 1.8239 - accuracy: 0.4240 - val_loss: 1.6327 - val_accuracy: 0.4069
Epoch 3/100
203/203 [==============================] - 50s 244ms/step - loss: 1.7281 - accuracy: 0.4411 - val_los

KeyboardInterrupt: ignored